In [1]:
# Third-party libraries
from bs4 import BeautifulSoup
import bs4 as bs
import urllib.request
import requests
import pandas as pd
import numpy as np
import json
from functools import reduce

In [2]:
url = "https://www.forsyth.k12.ga.us/Page/52982"

In [3]:
# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

In [4]:
# Parse the html content
soup = BeautifulSoup(html_content, "html.parser")
# print(soup.prettify()) # print the parsed data of html

In [5]:
covid_links = {}
for link in soup.find_all("a"):
    if str(link.text).lower().find('week ') != -1:
        covid_links[str(link.text)] = str(link['href']).replace('s','',1)
    else:
        pass

print(covid_links)

{'Week 1': 'http://www.forsyth.k12.ga.us/Page/53161', 'Week 2': 'http://www.forsyth.k12.ga.us/Page/53166', 'Week 3': 'http://www.forsyth.k12.ga.us/Page/53167'}


In [6]:
covid_url = covid_links['Week 2']
covid_url

'http://www.forsyth.k12.ga.us/Page/53166'

In [7]:
source = requests.get('http://www.forsyth.k12.ga.us/Page/53166').text
soup_wk1 = bs.BeautifulSoup(source,'html.parser')

In [8]:
token = soup_wk1.find_all('input', id=lambda x: x and x.startswith('hidData'))

In [9]:
info = [x.get('value') for x in token][0]

In [10]:
replace_list = ['<p>','</p>','<p class=\\"\\', '>']
for t in replace_list:
    info = info.replace(t, '')
info = info.replace('""', '"')
print(info)

[["School Name","Total Staff & Student","8/17/20","8/18/20","8/19/20","8/20/20","8/21/20"],["ELEMENTARY",null,null,null,null,null,null],["Big Creek ES","706","0","0","0","0","0"],["Brandywine ES","1,357","0","0","0","0","1"],["Brookwood ES","1,078","0","0","0","0","0"],["Chattahoochee ES","850","0","2","0","0","0"],["Chestatee ES","1,381","0","0","1","0","0"],["Coal Mountain ES","821","0","0","2","0","0"],["Cumming ES","1,055","0","0","0","0","0"],["Daves Creek ES","1,285","0","0","0","0","0"],["Haw Creek ES","1,131","0","0","0","0","0"],["Johns Creek ES","1,103","0","0","0","0","0"],["Kelly Mill ES","1,257","0","0","0","0","0"],["Mashburn ES","781","0","0","0","0","0"],["Matt ES","1,125","0","0","0","0","0"],["Midway ES","961","0","0","0","0","0"],["Poole's Mill ES","807","0","0","0","1","0"],["Sawnee ES","1,459","0","0","0","0","0"],["Settles Bridge ES","1,099","0","0","0","0","0"],["Sharon ES","999","0","0","0","0","0"],["Shiloh Point ES","1,452","0","0","0","0","1"],["Silver City E

In [11]:
info_json = json.loads(info)

In [12]:
headers = info_json.pop(0)
df = pd.DataFrame(info_json, columns=headers)
df.head()

,School Name,Total Staff & Student,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
0,ELEMENTARY,None,None,None,None,None,None
1,Big Creek ES,706,0,0,0,0,0
2,Brandywine ES,"1,357",0,0,0,0,1
3,Brookwood ES,"1,078",0,0,0,0,0
4,Chattahoochee ES,850,0,2,0,0,0


In [13]:
columns = df.columns
columns

Index(['School Name', 'Total Staff & Student', '8/17/20', '8/18/20', '8/19/20',
       '8/20/20', '8/21/20'],
      dtype='object')

In [14]:
for col in columns[1:]:
    df[col] = pd.to_numeric(df[col].str.replace(',',''), errors='coerce')

In [15]:
df.head()

,School Name,Total Staff & Student,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
0,ELEMENTARY,NaN,NaN,NaN,NaN,NaN,NaN
1,Big Creek ES,706.0,0.0,0.0,0.0,0.0,0.0
2,Brandywine ES,1357.0,0.0,0.0,0.0,0.0,1.0
3,Brookwood ES,1078.0,0.0,0.0,0.0,0.0,0.0
4,Chattahoochee ES,850.0,0.0,2.0,0.0,0.0,0.0


In [16]:
df.replace(to_replace=['None'], value=np.nan, inplace=True)
df.replace(to_replace=[','], value=[''], inplace=True)
df.dropna(subset=['Total Staff & Student'], inplace=True)

In [17]:
df.head()

,School Name,Total Staff & Student,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
1,Big Creek ES,706.0,0.0,0.0,0.0,0.0,0.0
2,Brandywine ES,1357.0,0.0,0.0,0.0,0.0,1.0
3,Brookwood ES,1078.0,0.0,0.0,0.0,0.0,0.0
4,Chattahoochee ES,850.0,0.0,2.0,0.0,0.0,0.0
5,Chestatee ES,1381.0,0.0,0.0,1.0,0.0,0.0


In [18]:
split_names = df['School Name'].str.rsplit(' ', 1, expand=True)

In [19]:
split_names

,0,1
1,Big Creek,ES
2,Brandywine,ES
3,Brookwood,ES
4,Chattahoochee,ES
5,Chestatee,ES
6,Coal Mountain,ES
7,Cumming,ES
8,Daves Creek,ES
9,Haw Creek,ES
10,Johns Creek,ES


In [20]:
df['school'] = split_names[0]
df['type'] = split_names[1]

In [21]:
df.drop(['School Name'], axis=1, inplace=True)
cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]
df = df[cols]
df.head()

,school,type,Total Staff & Student,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
1,Big Creek,ES,706.0,0.0,0.0,0.0,0.0,0.0
2,Brandywine,ES,1357.0,0.0,0.0,0.0,0.0,1.0
3,Brookwood,ES,1078.0,0.0,0.0,0.0,0.0,0.0
4,Chattahoochee,ES,850.0,0.0,2.0,0.0,0.0,0.0
5,Chestatee,ES,1381.0,0.0,0.0,1.0,0.0,0.0


In [22]:
def get_urls(url):
    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text
    # Parse the html content
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Getting weekly number lists
    covid_links = []
    for link in soup.find_all("a"):
        if str(link.text).lower().find('week ') != -1:
            # replacing https with http and getting link url
            covid_links.append(str(link['href']).replace('s','',1))
        else:
            pass
    
    return covid_links

In [23]:
def get_weekly_info(url):   
    source = requests.get(url).text
    soup = bs.BeautifulSoup(source,'html.parser')
    
    # finding hidData element
    token = soup.find_all('input', id=lambda x: x and x.startswith('hidData'))
    # getting 'value' from inside tag
    info = [x.get('value') for x in token][0]
    
    return info

In [24]:
info = get_weekly_info("http://www.forsyth.k12.ga.us/Page/53161")

In [25]:
def info_dataframe(info):
    
    # Removing html characters
    replace_list = ['<p>','</p>','<p class=\\"\\', '>']
    for t in replace_list:
        info = info.replace(t, '')
        
    # Removing extra ""
    info = info.replace('""', '"')
    
    # Replacing data for Alliance Academy
    info = info.replace('Innovation', 'Innovation HS')
    
    # Reading info as json string
    info_json = json.loads(info)
    
    # Removing headers
    headers = info_json.pop(0)
    
    # Reading into dataframe
    df = pd.DataFrame(info_json, columns=headers)
    
    return df

In [26]:
def clean_df(df):
    # Removing , fron numbers and converting to numeric values
    columns = df.columns
    for col in columns[1:]:
        df[col] = pd.to_numeric(df[col].str.replace(',',''), errors='coerce')
    
    # Replacing none values with nulls
    df.replace(to_replace=['None'], value=np.nan, inplace=True)
    # Dropping rows with nulls
    df.dropna(subset=['Total Staff & Student'], inplace=True)
    
    # Splitting school name into name and category
    split_names = df['School Name'].str.rsplit(' ', 1, expand=True)
    # Creating columns for name and category
    df['school'] = split_names[0]
    df['type'] = split_names[1]
    
    # Final cleaning and ordering of columns
    df.drop(['School Name'], axis=1, inplace=True)
    cols = df.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    df = df[cols]
    
    return df

In [27]:
def get_weekly_dataframes(master_url):
    
    # Getting weekly links
    url_links = get_urls(master_url)
    
    # Getting dataframe for each week
    df_list = []
    for url in url_links:
        info = get_weekly_info(url)
        df = info_dataframe(info)
        df = clean_df(df)
        df_list.append(df)
        
    return df_list

In [28]:
def merge_dataframes(df_list):
    df_final = reduce(lambda x, y: pd.merge(x, y, on = ['school', 'type', 'Total Staff & Student']), df_list)
    print(df_final.head())
    print(df_final.shape)
    
    return df_final

In [29]:
def get_covid_data(url):
    df_list = get_weekly_dataframes(url)
    df_final = merge_dataframes(df_list)
    
    # Columns to sum
    df_columns = df_final.columns
    df_columns = df_columns[3:]
    
    # Adding sum column
    df_final['total_positive'] = df_final[df_columns].sum(axis=1)
    df_final['percent_positive'] = (df_final['total_positive'] / df_final['Total Staff & Student']) * 100
    
    # Cleaning type data
    df_final['type'] = df_final['type'].str.strip()
    
    # Dropping TOTAL row
    df_final = df_final[df_final['school'] != 'TOTAL']
    
    return df_final

In [30]:
df_final = get_covid_data("https://www.forsyth.k12.ga.us/Page/52982")

KeyError: ['Total Staff & Student']

In [ ]:
df_final.head(50)

In [ ]:
total_positive = df_final['total_positive'].sum()
total_positive

In [ ]:
columns = df_final.columns
daily_columns = columns[3:-3]

daily_results = df_final[daily_columns]
daily_results

daily_totals = daily_results.sum()
daily_totals

In [ ]:
daily_totals.index[-1]

In [ ]:
import plotly.graph_objects as go

fig_indicator = go.Figure()

fig_indicator.add_trace(go.Indicator(
    mode = "number+delta",
    value = int(daily_totals[-1]),
    title = {"text": f"Daily Cases ({daily_totals.index[-1]})<br><span style='font-size:0.8em;color:gray'>Increase from {daily_totals.index[-2]}</span>"},
    delta = {'reference': int(daily_totals[-1]), 'relative': True, 'position' : "bottom"}))

fig_indicator.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = int(daily_totals.sum()),
    title = {"text": f"Total Cases ({daily_totals.index[-1]})<br><span style='font-size:0.8em;color:gray'>Increase from {daily_totals.index[-2]}</span>"},
    delta = {'reference': int(daily_totals[:-1].sum()), 'relative': True, 'position' : "bottom"}))

fig.show()

In [ ]:
df_final['type'] = df_final['type'].str.strip()
df_grouped = df_final.groupby('type')

In [ ]:
df_grouped_cols = df_grouped[daily_columns].sum()
df_grouped_cols
df_grouped_yesterday = df_grouped_cols.iloc[:,:-1]
df_grouped_yesterday
# df_grouped_cols

In [ ]:
type_grouped = df_grouped_cols.sum(axis=1)
type_grouped
type_grouped_yesterday = df_grouped_yesterday.sum(axis=1)
type_grouped_yesterday

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = int(type_grouped[0]),
#     title = {"text": f"Elementary School Total"},
    domain = {'x': [0, 0.4], 'y': [0, 0.25]},
    align = "center",
    delta = {'reference': int(type_grouped_yesterday[0]), 'relative': True, 'position' : "right"}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = int(type_grouped[2]),
#     title = {"text": f"Middle School Total"},
    domain = {'x': [0, 0.4], 'y': [0.36, 0.62]},
    align = "center",
    delta = {'reference': int(type_grouped_yesterday[2]), 'relative': True, 'position' : "right"}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = int(type_grouped[1]),
#     title = {"text": f"High School Total"},
    domain = {'x': [0, 0.4], 'y': [0.75, 1]},
    align = "center",
    delta = {'reference': int(type_grouped_yesterday[1]), 'relative': True, 'position' : "right"}))

fig.update_layout(
    title={"text": "<span style='font-size:4em;color:black'>Total Cases</span>", 
           "yanchor":"middle",
           "xanchor":"center",
           "x":0.4,
           "y":0.85})

fig.update_layout(
    annotations=[
        dict(text="<span style='font-size:4em;color:black'>High School</span>",
             align="left",
            showarrow=False,
            x=0.72,
            y=0.95,),
        dict(text="<span style='font-size:4em;color:black'>Middle School</span>",
             align="left",
            showarrow=False,
            x=0.75,
            y=0.48,),
        dict(text="<span style='font-size:4em;color:black'>Elementary</span>",
             align="left",
            showarrow=False,
            x=0.7,
            y=0.05,)])

fig.show()

In [ ]:
df_sums = df_final[daily_columns].sum()
df_aggregated = df_sums.cumsum()

In [ ]:
df_aggregated

In [ ]:
import plotly.graph_objects as go
import numpy as np

title = 'Total Cases by Date'
colors = ['rgb(83,190,171)']

mode_size = [8]
line_size = [2]

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_aggregated.index, y=df_aggregated, mode='lines',
    name=labels[0],
    line=dict(color=colors[0], width=2),
    connectgaps=True,
))

# endpoints
fig.add_trace(go.Scatter(
    x=[df_aggregated.index[-1]],
    y=[df_aggregated[-1]],
    mode='markers',
    marker=dict(color=colors[0], size=6)
))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=True,
        showticklabels=True,
        tickfont=dict(
        family='Arial',
            size=12,
            color='rgb(82, 82, 82)')
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=20,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

fig.update_layout(annotations=[dict(xref='paper', yref='paper', x=0.5, y=0.9,
                              xanchor='center', yanchor='bottom',
                              text=title,
                              font=dict(size=26,
                                        color='rgb(82, 82, 82)'),
                              showarrow=False)])


fig.show()

In [ ]:
school_groups = df_final.groupby('school')
school_sums = school_groups['total_positive'].sum()
school_sums = school_sums.sort_values(ascending=False)
top_schools = school_sums[:10].sort_values(ascending=True)
top_schools

In [ ]:
import plotly.graph_objects as go

title = "Schools With Most Confirmed Cases"

fig = go.Figure(go.Bar(
            x=top_schools,
            y=top_schools.index,
            orientation='h',
            marker=dict(
                color='rgb(83,190,171)')))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=True,
        showticklabels=True,
        tickfont=dict(
        family='Arial',
            size=12,
            color='rgb(82, 82, 82)')
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=100,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

fig.update_layout(annotations=[dict(xref='paper', yref='paper', x=0.5, y=1,
                              xanchor='center', yanchor='bottom',
                              text=title,
                              font=dict(size=26,
                                        color='rgb(82, 82, 82)'),
                              showarrow=False)])

fig.show()